# CDS tutorial extended _ exploration of peculiar galaxies

Stefania Amodeo¹, Katarina A. Lutz¹

1. Université de Strasbourg, CNRS, Observatoire Astronomique de Strasbourg, UMR 7550, F-67000, Strasbourg, France

***

## Introduction

In this tutorial, we will explore Arp's Catalog of peculiar galaxies and check wich of them are in both SDSS and GALEX surveys.

In [ ]:
# Query databases
from astroquery.vizier import Vizier
from astroquery.cds import cds
import pyvo

# Atronomical data handling
from astropy.coordinates import SkyCoord
import astropy.units as u

# Visualization
import ipyaladin.aladin_widget as ipyal

# Manipulate MOCS
import mocpy


## Find and download tables from VizieR

<img src=Data/images/logos/vizier.svg alt="vizier's logo" style="height:100px; display: inline-block;"/>
We use the `Vizier` sub-module from the `astroquery` module. 

We will explore the [Arp's Catalog of Peculiar Galaxies](http://arpgalaxy.com/) compiled by [Halton C. Arp](https://www.haltonarp.com/) and published by Denis Webb.

Hence, we ask `Vizier` to find all catalogues that have a match with the keywords 'Arp Webb' and write the result into the variable `catalog_list_arp` (first line of code). Then we tell Python to `print` out the query result in a readable way (second line of code).  

In [ ]:
catalog_list_arp = Vizier.find_catalogs('Arp')
for k, v in catalog_list_arp.items():
    print(k, ': ', v.description)


Alternatively you can also search for the table with `pyvo` using the Table Access Protocol (TAP) to query VizieR. In this case, you first need to define the TAP endpoint of VizieR and then send a query written in ADQL (an SQL-like query language optimised for astronomy) to this endpoint. Finally we again print out the result of this query.

In [ ]:
tap_vizier = pyvo.dal.TAPService(
    'https://tapvizier.cds.unistra.fr/TAPVizieR/tap')
query = """SELECT  *  FROM tap_schema.tables 
               WHERE table_name LIKE '%arp%' """
catalog_list_arp = tap_vizier.search(query).to_table()
catalog_list_arp['table_name', 'description']


The catalogue that we are interested in today is `VII/192` and its description is "Arp's Peculiar Galaxies (Webb 1996)". In the TAP query we already see that there are two tables in this catalogue: `VII/192/arplist` and `VII/192/arpord`. To be able to work with these data, let's load them into this notebook. 

Again we can use either `astroquery` or `pyvo`. Which one you chose is up to you and which one you like better.

In oder to get the full catalogue with `astroquery`, we first set the row limit to infinite (i.e. `-1` in the notation of this package) and then ask `Vizier` to write the content of the catalogue into the variable `catalogs_arp`:

In [ ]:
catalogs_arp = Vizier(row_limit=-1).get_catalogs('VII/192')


Now let's inspect, what we got:

In [ ]:
print(catalogs_arp)


As we have seen in the first part of the tutorial, the catalogue "Arp's Peculiar Galaxies (Webb 1996)" comes with two tables: arpord and arplist. As you can see from the printout `Vizier` has downloaded both of them. However, we are still only interested in the arplist (index `1` in the TableList). Therefore, we make Python write only the arplist table into a new variable `table_arplist` (first line) and then display the table (second line):

In [ ]:
table_arplist = catalogs_arp[1]
table_arplist


Nice, we got the data we want. Before moving on, let's have a look at how to use `pyVO` to get the same table. Just for safety we define again the VizieR TAP endpoint. It is the same one as above, but if the cells of this notebook are not run one after another, this safety measure ensure we always query the right service.

Then we write a simple query telling VizieR that we want all the rows and all `*` the columns from the table `VII/192/arplist`. Note that we have to encompass the table name with quotation marks due to the special character `/` in the table name. 

In [ ]:
tap_vizier = pyvo.dal.TAPService(
    'https://tapvizier.cds.unistra.fr/TAPVizieR/tap')
query = """SELECT  *  FROM "VII/192/arplist" """
table_arplist = tap_vizier.search(query).to_table()
table_arplist


There we go, we now know two nifty ways to get VizieR tables with Python. Obviously there are many other things you can do with this tools. For example if you set your TAP endpoint to `https://simbad.cds.unistra.fr/simbad/sim-tap` instead of the VizieR enpoint, you can query SIMBAD just like that. 

<img src=Data/images/logos/aladin.svg alt="aladin's logo" style="height:100px; display: inline-block;"/>
Note that not all rows but only the first and last 10 rows for all columns are shown in the display. To finish off this little excursion, we now want to visualise the location of the entries of this table in an Aladin Lite widget. To do so, we tell Python to take the variable `aladin` and add the table `table_arplist` to it:

In [ ]:
aladin = ipyal.Aladin()
aladin


In [ ]:
aladin.add_table(table_arplist)


Now explore the Aladin Lite widget. You will find that the location of the sources in the arplist table are marked with coloured symbols. You can zoom in and out to look at the different sources and check their peculiarity. If you click on one of the colour symbols, you will be able to see the corresponding row in the bottom of the Aladin Lite widget. 

Note if you are using Jupyter Lab instead of single Jupyter notebooks, you can open two notebooks that share a kernel. Then one notebook could contain your working environment, where you get and work on your tables ect. The other notebook could contain the Aladin Lite widget, and just remain visible all the time. This way there would be no need to scroll up and down. 

## MOCs in Python

We can use MOCs to find out whether any of these galaxies have been observed both by SDSS and GALEX. For this task we use the `cds` module of `astroquery` and in the functionalities of `MOCpy` to get the intersection of the MOCs. First let's query the MOC server for all things SDSS and all things GALEX.

In [ ]:
info_sdss = cds.find_datasets(meta_data="ID=*SDSS*")
info_sdss[['ID']]


In [ ]:
info_galex = cds.find_datasets(meta_data="ID=*GALEX*")
info_galex[['ID']]


Now that we know what the relevant data sets are called, we can move on to download the corresponding MOCs,

In [ ]:
moc_sdss = cds.find_datasets(meta_data="ID=CDS/P/SDSS9/color", return_moc=True)
moc_galex = cds.find_datasets(
    meta_data="ID=CDS/P/GALEXGR6/AIS/color", return_moc=True)


and calculate the intersection of these two MOCs.

In [ ]:
moc_intersection = moc_sdss.intersection(moc_galex)
print(
    f'The intersection of SDSS and GALEX covers {round(moc_intersection.sky_fraction *100., 1)} % of the sky')


Now we can use this MOC to filter our table of Peculiar Galaxies. 

In [ ]:
coords = SkyCoord(ra=table_arplist['RAJ2000'],
                  dec=table_arplist['DEJ2000'],
                  unit=(u.deg, u.deg))
#mask = moc_intersection.contains_skycoords(coords)
# print(
#    f'{len(table_arplist[mask])} peculiar galaxies are observed by both GALEX and SDSS')


To visualise only the galaxies within the MOC, we can add the filtered table to the AladinLite widget. This table will show up in a different colour than the first one we visualised.

In [ ]:
# aladin.add_table(table_arplist[mask])


The last thing to do with our MOC is to visualise it. We can either plot it using `matplotlib` (see the MOCpy documentation, how to do this) or add it to our AladinLite widget:

In [ ]:
aladin.add_moc_from_dict(moc_intersection.serialize(format='json'),
                         {'color': 'red', 'opacity': 0.45})


Before leaving the tutorial, don't forget to scroll back up and look at the results in the Aladin widget ;)